In [4]:
import sys
try:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")
except FileNotFoundError:
    pass

import pandas as pd
import numpy as nm
import sys

import ufal.udpipe as ud
from ufal.udpipe import Model, Pipeline, ProcessingError
import pyconll, pyconll.util
from ufal.udpipe import Model, Pipeline
import os
import re
import wget

In [5]:
subj  = {"nsubj", "nsubj:pass"}
obj   = {"iobj", "obj"}
names = {"PRON", "PROPN"}

## Process with UDPIPE

In [6]:
def process(text):
    processed = []
    text = text.split("\n")[3:]
    for word in text[1:]:
        word = word.split("\t")
        if(len(word) == 10):
            processed += [word[:4] + word[6:8]]
    return processed


def tag_ud(text, modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    
    items = []
    for line in text:
        output = process_pipeline.process(line)
        items += [process(output)]
    return items

## Change for PersonX

In [7]:
def place_persons(item):
    subj_parent = -1
    for word in item:
        if((word[5] in subj) and (word[3] in names)):
            word[1] = "ЧеловекX"
            word[2] = "ЧеловекX"
            subject_parent = word[4]
    if(subj_parent != -1):
        for word in item:
            if((word[5] in obj) and (word[3] in names) and word[4] == subj_parent):
                word[1] = "ЧеловекY"
                word[2] = "ЧеловекY"

In [9]:
udpipe_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
modelfile = "udpipe_syntagrus.model"
#textfile = wget.download(text_url)

text = ["Мастер увидел Маргариту с жёлтыми тюльпанами. Он с замиранием сердца заметил её в толпе. Доброго Диму использует злой Петя"]

source = tag_ud(text=text, modelfile=modelfile)
s1 = source[0]
place_persons(s1)
print(s1)

[['1', 'ЧеловекX', 'ЧеловекX', 'PROPN', '2', 'nsubj'], ['2', 'увидел', 'видеть', 'VERB', '0', 'root'], ['3', 'Маргариту', 'Маргарита', 'PROPN', '2', 'obj'], ['4', 'с', 'с', 'ADP', '6', 'case'], ['5', 'жёлтыми', 'желтый', 'ADJ', '6', 'amod'], ['6', 'тюльпанами', 'тюльпан', 'NOUN', '2', 'obl'], ['7', '.', '.', 'PUNCT', '6', 'punct'], ['1', 'ЧеловекX', 'ЧеловекX', 'PRON', '5', 'nsubj'], ['2', 'с', 'с', 'ADP', '3', 'case'], ['3', 'замиранием', 'замирание', 'NOUN', '5', 'obl'], ['4', 'сердца', 'сердце', 'NOUN', '3', 'nmod'], ['5', 'заметил', 'замечать', 'VERB', '0', 'root'], ['6', 'её', 'она', 'PRON', '5', 'obj'], ['7', 'в', 'в', 'ADP', '8', 'case'], ['8', 'толпе', 'толпа', 'NOUN', '5', 'obl'], ['9', '.', '.', 'PUNCT', '8', 'punct'], ['1', 'Доброго', 'добрый', 'ADV', '3', 'advmod'], ['2', 'Диму', 'Дима', 'PROPN', '3', 'iobj'], ['3', 'использует', 'использовать', 'VERB', '0', 'root'], ['4', 'злой', 'злой', 'ADJ', '5', 'amod'], ['5', 'ЧеловекX', 'ЧеловекX', 'PROPN', '3', 'nsubj']]
